# ForumQA 格式的心理咨询论坛问答语料

为 NVIDIA/Megatron-LM 准备该领域的 finetune 数据集

## 要点

- 使用教少量的心理咨询论坛问答数据，在之前的泛情绪类预训练模型基础上进行 finetune
- 我们自己规定的 ForumQA 格式和我们按照这个格式修改的训练

论坛 QA 包括：

* 问题标题
* 问题正文
* 问题类型标签列表
* 回答列表

这个数据集是一个 Infinit(Iterable-style) datasets，他将问题与类型、回答反复组合，循环迭代，返回给 Dataloader

数据文件是 JSON lines 格式，每一行都是 JSON Object,其格式形如::

    {
        title: "...",
        text: "...",
        tags: ["...", "..."],  # Optional
        answers: [
            {text: "..."},
            {text: "..."},
            # ...
            {text: "..."},
        ]
    }

返回的数据是 token ID 的一维数组，形如::

    xxxxx<sep>xxxxxx[<sep>xxxx]<sep><sep>[bos]xxxxxxxxx[eos]
    Title      Text      tag                    Answer

## Importings

In [1]:
import os
import json
import random
from copy import copy
from contextlib import closing, ExitStack
from functools import partial
from glob import glob, iglob
from fileinput import FileInput
from itertools import chain, repeat
from multiprocessing import Pool

import pandas as pd
import sentencepiece as spm
from tqdm.auto import tqdm

from IPython.display import display

## Tokenizer

使用与 Pretrained-Model 同样的 tokenizer

In [2]:
spm_model_path = '../data/spm/gpt2_huamei_corpus_bpe_32k_v2.model'

tokenizer = spm.SentencePieceProcessor()
tokenizer.load(spm_model_path)

True

## Corpus

In [4]:
corpus_files = [
    '/home/liuxy/Public/yiren-scrapy-crawlers/data/[xinli001+jiandanxinli]-qa[191010].jsonl',
]


In [5]:
print('语料文件:')
display(corpus_files)
print()

it = FileInput(corpus_files)
total = sum(1 for _ in tqdm(it, unit='line'))

print(f'总行数: {total:,d}')

语料文件:


['/home/liuxy/Public/yiren-scrapy-crawlers/data/[xinli001+jiandanxinli]-qa[191010].jsonl']


总行数: 200,526


现在预览语料数据格式，随机选择一个：

In [6]:
n = random.randint(0, total)

it = FileInput(corpus_files)
for i, line in tqdm(zip(range(n), it), total=n):
    if i+1 < n:
        continue
    data = json.loads(line)
    display(i)
    display(data)


87418

{'id': 100037650,
 'url': 'https://www.xinli001.com/qa/100037650',
 'crawl_time': '2019-09-23 05:04:33',
 'user_url': 'https://www.xinli001.com/user/11393641',
 'answers': [{'page': 2,
   'user_url': 'https://www.xinli001.com/user/1001010141',
   'user_name': '王永馥',
   'user_title': '',
   'reward': None,
   'like_num': 1,
   'time': '2017-02-16',
   'comment_num': 0,
   'comments': [],
   'text': '说句你不爱听的，这男友也太自私了点，但是女生在被需要时感觉很有价值感，就对上了。'}],
 'title': '异地恋男友非要我去找他',
 'answer_num': 1,
 'time': '2017-02-16',
 'read_num': 607,
 'text': '他在外地上学 我还在家里。说实话我父母嫌他家里穷不让我和他在一起。可是我真的很喜欢他，我不管他穷不穷只要我们一起努力肯定饿不死。可是他上学去了天天问我怎么不去找他，我是真的不能去找他，我爸妈知道了非被我气死不可。我真的是不知道怎么办。我就想他在学校好好读书好好考研，可他天天就只会说想我让我快点去。哎~~~',
 'tags': ['婚姻']}

## 采样

### 过滤条件

我们应：

- 选取问题+回答构成输入样本
- 一个问题可以对应多个回答
- 不要太短的
- 不要太长的
- 只返回需要的属性

在此基础上，选用 `10,000` 个样本用于今次的开发

In [41]:
### 长短限制定义：

MIN_QUESTION_LENGTH = 16
MAX_QUESTION_LENGTH = 1024
MIN_ANSWER_LENGTH = 32
MAX_LENGTH = 2048


def filter_sample(line):
    line = line.strip()
    if not line:
        return None
    data = json.loads(line)
    # 裁减属性
    attrs = list(data.keys())
    for attr in attrs:
        if attr not in ('title', 'text', 'tags', 'answers'):
            del data[attr]
    #
    tags = data.get('tags', []) 
    # 没有主题的不要
#     if not tags:
#         return []
    # 问题:
    title = data['title']
    # 没有标题的不要
    if not title:
        return None
    text = data['text']
    # 没有问题内容的不要
    if not text:
        return []
    # 没有回答的不要
    if not data.get('answers'):
        return None
    n_title = len(tokenizer.encode_as_ids(title))
    n_text = len(tokenizer.encode_as_ids(text))
    # 太短不要
    if n_title + n_text < MIN_QUESTION_LENGTH:
        return None
    # 太长不要
    if n_title + n_text > MAX_QUESTION_LENGTH:
        return None
    # 答案长度检查
    answers = []
    for answer in data['answers']:
        text = answer['text']
        length = len(tokenizer.encode_as_ids(text))
        # 太短不要
        if length < MIN_ANSWER_LENGTH:
            continue
        # 太长不要
        if n_title + n_text + length > MAX_LENGTH:
            continue
        # 裁减属性
        attrs = list(answer.keys())
        for attr in attrs:
            if attr not in ('text',):
                del answer[attr]
        #
        answers.append(answer)
    # 答案过滤后没有了的不要
    if not answers:
        return None
    data['answers'] = answers
    #
    return data

用上一次随机取得的行试试看：

In [42]:
filter_sample(line)

### 过滤全部样本

In [43]:
samples = []

with FileInput(corpus_files) as fp, Pool() as pool:
    mapper = pool.imap_unordered(
        filter_sample,
        tqdm(fp, total=total),
        chunksize=64
    )
    samples = [
        d
        for d
        in tqdm(mapper, total=total)
        if d
    ]

print(f'满足过滤条件的样本个数: {len(samples):,d}/{total:,d} ({len(samples)/total*100}%)')




满足过滤条件的样本个数: 149,423/200,526 (74.51552417142913%)


> 💡 **Tips**:
>
> 我们可以反复调整过滤条件，直到满足的样本个数达到要求

### 划分数据集

更具过滤后的样本个数划分 train/val/test 数据集

In [44]:
parts = {
    'train': 140_000,
    'val': 5_000,
    'test': 4_000,
}

print('数据集划分：')
display(parts)

assert sum(parts.values()) <= len(samples)

数据集划分：


{'train': 140000, 'val': 5000, 'test': 4000}

### 随机/洗牌

随机种子:

In [48]:
%%time

seeds = []
seeds.extend(chain.from_iterable(
    repeat(k, v) for k, v in parts.items()
))
seeds.extend(repeat(None, len(samples)-sum(parts.values())))
random.shuffle(seeds)
random.shuffle(samples)

assert len(seeds) == len(samples)

CPU times: user 314 ms, sys: 0 ns, total: 314 ms
Wall time: 312 ms


### 保存结果

#### 输出文件路径

保存为我们自己规定的 ForumQA Loose Json 格式


In [50]:
%%time

output_dir = '../data/xinliqa'

os.makedirs(output_dir, exist_ok=True)

output_files = {
    k: os.path.join(output_dir, f'{k}.json')
    for k in parts.keys()
}

print('输出文件:')
display(output_files)

输出文件:


{'train': '../data/xinliqa/train.json',
 'val': '../data/xinliqa/val.json',
 'test': '../data/xinliqa/test.json'}

CPU times: user 4.95 ms, sys: 2.52 ms, total: 7.47 ms
Wall time: 4.74 ms


#### 写目标文件

In [51]:
lines_iter = FileInput(corpus_files)

with ExitStack() as stack:
    files = {
        k: stack.enter_context(open(v, 'w'))
        for k, v in output_files.items()
    }
    for seed, sample in tqdm(zip(seeds, samples), total=len(samples)):
        fp = files.get(seed)
        if not fp:
            continue
        print(
            json.dumps(sample, ensure_ascii=False),
            file=fp
        )

检查输出文件

In [52]:
s = ' '.join(output_files.values())

!wc -l {s}
print()

!du -h {s}
print()

   140000 ../data/xinliqa/train.json
     5000 ../data/xinliqa/val.json
     4000 ../data/xinliqa/test.json
   149000 总用量

248M	../data/xinliqa/train.json
8.7M	../data/xinliqa/val.json
6.9M	../data/xinliqa/test.json

